## Oil blending problem

A refinery has to blend 4 types of oil to obtain 3 types of gasoline. The following table reports the available quantity of oil of each type (in barrels) and the respective unit cost (Euro per barrel)

Oil type|Cost|Availability
--------|----|------------
1|9|5000
2|7|2400
3|12|4000
4|6|1500


Blending constraints are to be taken into account, since each type of gasoline must contain at least a specific, predefined, quantity of each type of oil, as indicated in the next table. The unit revenue of each type of gasoline (Euro per barrel) is also indicated

Gasoline type|Requirements|Revenue
-------------|------------|-------
A|$\geq$ 20% of type 2| 12
A|$\leq$ 30% of type 3|12
B|$\geq$ 40% of type 3|18
C|$\leq$ 50% of type 2|10


In [17]:
import mip

In [18]:
# Set of oil types
I = [1, 2, 3, 4]

# Set of gasoline types
J = ["A", "B", "C"]

# Unit cost for oil of type i
c = [9, 7, 12, 6]

# Availability of oil type i
a = [5000, 2400, 4000, 1500]

# Price of gasoline of type j
r = [12, 18, 10]

# Maximum quantity (percentage) of oil
q_max = {}
for i in I:
  for j in J:
    q_max[(str(i),j)] = 1

q_max[('2','A')] = 0.3
q_max[('1','C')] = 0.5

# Minimum quantity (percentage) of oil
q_min = {}
for i in I:
  for j in J:
    q_min[(str(i),j)] = 0

q_min[('1','A')] = 0.2
q_min[('2','B')] = 0.4

In [19]:
# Define a model
model = mip.Model()

# Define variables
x = [[model.add_var(name=f"X{i}{j}") for j in J] for i in I]
y = [model.add_var(name=f"N{j}") for j in J]

# default --> variabili reali, lower bound = 0 e upper bound = +INF --> edita a seconda di ciò che ti serve

In [20]:
# Define the objective function
model.objective = mip.maximize(
  mip.xsum([r[j] * y[j] for j, gas in enumerate(J)])
  -
  mip.xsum([c[i] * mip.xsum([x[i][j] for j in range(len(J))]) for i in range(len(I))])
)

In [21]:
# CONSTRAINTS
# Availability constraint
for i in range(len(I)):
  model.add_constr(mip.xsum([x[i][j] for j in range(len(J))]) <= a[i])

# Conservation constraint
for j in range(len(J)):
  model.add_constr(mip.xsum([x[i][j] for i in range(len(I))]) == y[j])

# Maximum quantity
for i, oil in enumerate(I):
  for j, gas in enumerate(J):
    model.add_constr(x[i][j] <= q_max[(str(oil), gas)] * y[j])

# Minimum quantity
for i, oil in enumerate(I):
  for j, gas in enumerate(J):
    model.add_constr(x[i][j] >= q_min[(str(oil), gas)] * y[j])

In [22]:
# Optimizing command
model.optimize()

Clp0024I Matrix will be packed to eliminate 10 small elements
Coin0506I Presolve 21 (-10) rows, 15 (0) columns and 55 (-10) elements
Clp0000I Optimal - objective value 72000
Coin0511I After Postsolve, objective 72000, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 72000 - 10 iterations time 0.002, Presolve 0.00
Starting solution of the Linear programming problem using Dual Simplex



<OptimizationStatus.OPTIMAL: 0>

In [23]:
# Optimal objective function value
round(model.objective.x, 2)

72000.0

In [24]:
for var in model.vars:
  print(var.name, "\t=\t", round(var.x, 2))

X1A 	=	 5000.0
X1B 	=	 0.0
X1C 	=	 0.0
X2A 	=	 0.0
X2B 	=	 2400.0
X2C 	=	 0.0
X3A 	=	 1900.0
X3B 	=	 2100.0
X3C 	=	 0.0
X4A 	=	 0.0
X4B 	=	 1500.0
X4C 	=	 0.0
NA 	=	 6900.0
NB 	=	 6000.0
NC 	=	 0.0
